In [ ]:
!pip install tensorflow-addons
!pip install tqdm

# **Importing Modules & Downloading Data**

---



In [ ]:
import os
import gc
import math
import random
import scipy.io
import cv2 as cv
import numpy as np
import scipy as sp
import urllib.request
from PIL import Image
import tensorflow as tf
from random import randint
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from packaging import version
import tensorflow_addons as tfa
from tqdm import tqdm

print(tf.__version__)
print(np.__version__)
print(tf.keras.__version__)


In [ ]:
if not os.path.isfile("check_file"):
  !rm -rf sample_data
  !wget --no-check-certificate -O keypoint-5.zip "https://onedrive.live.com/download?cid=82C508E2119FA196&resid=82C508E2119FA196%218384&authkey=AILIjeUiynP-fv0"
  !unzip keypoint-5.zip
  !rm -f keypoint-5.zip
  !touch check_file

# **Configurations**

---



In [ ]:
HEATMAP_SIZE = [40,30]
kp = None
annotator = 1 ## 0 for avg, 1-3 for other annotators
path_index = 0 ## 0 BED, 1 CHAIR, 2 SOFA, 3 SWIVELCHAIR,4 TABLE
hmaps = "KDE" ## the heatmap label type GAUSSIAN ## NORMAL ## SCORE ## KDE
epochs = 30

In [ ]:
w_ct = 4
if path_index == 2 or path_index == 3:
  w_ct = 6
  
if path_index == 2:
  kp = 14
elif path_index == 3:
  kp = 13
elif path_index == 4:
  kp = 8
else:
  kp = 10

# **Data Cleaning & Preprocessing**

---



In [ ]:
dir_path = os.getcwd()
categories = ["bed","chair","sofa","swivelchair","table"]
paths=[]
for category in categories:
    paths.append(os.path.join(dir_path,category))

In [ ]:
def avg_keypoint(points:list)->int:
    
    sums=[]
    for point in points:
        if point!=0 and not np.isnan(np.sum(point)):
            sums.append(point)
    try:        
        return sum(sums)/len(sums)
    except ArithmeticError:
        return -1
        

In [ ]:
def get_keypoints(matrix,avg=True,annotator=0):
    x_major = []
    y_major = []
    for i in range(matrix.shape[3]):
        x_minor=[]
        y_minor=[]
        for p in range(matrix.shape[1]):
            
            if avg:
                x = avg_keypoint(matrix[0,p,:,i])
                y = avg_keypoint(matrix[1,p,:,i])
            else:
                x = matrix[0,p,annotator,i]
                y = matrix[1,p,annotator,i]    
                if np.isnan(np.sum(x)) or np.isnan(np.sum(y)):
                    x, y = -1,-1
                    
            x_minor.append(int(x))
            y_minor.append(int(y))
                
        if len(x_minor)!=0 and len(y_minor)!=0:
            x_major.append(x_minor)
            y_major.append(y_minor)
            
    return np.array([x_major,y_major]).transpose(1,2,0)#examples,keypoints,(x,y)

In [ ]:
def local_contrast_normalize(image,kernel_shape=(1,3,9,9),sigma=2):
    """
    
    Kernel_shape: tuple contains following (out_chan,in_chan,height,width)
    image: nd numpy array with shape (height,width,channels)
    
    """
    
    height,width,channels = image.shape
    x = np.zeros(kernel_shape,dtype="float32")
    Z = 1/(2 * np.pi * sigma ** 2)
    mid = int(kernel_shape[-1]/2)
    
    for channel in range(kernel_shape[1]):
        for row in range(kernel_shape[2]):
            for column in range(kernel_shape[3]):
                coords = -((column-mid)**2 + (row-mid)**2)            
                x[0,channel,row,column] = Z*np.exp(coords/(2*sigma**2))
                
    kernel = (x/np.sum(x)).transpose(2,3,1,0) # kernel shape is (h,w,chan_in,chan_out)
    image = image.reshape((1,height,width,channels)).astype('float32')#image shape is (batch_size,h,w,channels)
    
    paddings = tf.constant([[0,0],[8,8],[8,8],[0,0]])
    padded = tf.pad(tf.constant(image),paddings,"CONSTANT")

    mid = int(np.floor(kernel.shape[1] / 2.))
    blurred = tf.nn.conv2d(padded,tf.constant(kernel),[1,1,1,1],"VALID")
    
    centered = (image-blurred[:,mid:-mid,mid:-mid,:]).numpy()

    normalized = ((centered-centered.min())/(centered.max()-centered.min()))
    
    return np.squeeze(normalized)

In [ ]:
def get_data(path,train=True,size=(320,240),lcn=False):
    
    if train:
        data_path = os.path.join(path,"train.txt")
    else:
        data_path = os.path.join(path,"test.txt")

    data=[]
    with open(data_path) as file:
        images_path = file.readlines()
        
        for image_path in images_path:
            image_path = image_path.split("/")[-1]
            
            full_path = os.path.join(path,"images",image_path)
            
            if full_path[-1] == "\n":
                full_path = full_path[:-1]

            image = plt.imread(full_path)
            if len(image.shape)<3:
              image = cv.cvtColor(image,cv.COLOR_GRAY2RGB)

            if lcn:
                image = local_contrast_normalize(image)
            
            while image.shape[0]<size[1]-20:
              image = cv.pyrUp(image)
            while image.shape[0]>size[1]+20:
              image = cv.pyrDown(image)
            image = cv.resize(image,size)
            
            
            norm  = (image-image.min())/(image.max()-image.min())
            data.append(norm)
            
    return data

In [ ]:
def get_labels(path,category,train=True):
   
    if train:
        data_path = os.path.join(path,"train.txt")
    else:
        data_path = os.path.join(path,"test.txt")
        
    labels = []
    
    with open(data_path) as file:
        images_path = file.readlines()

        for image_path in images_path:
            
            full_path = image_path.split("/")[-1]## number of image.jpg
            number = full_path.split(".")[0]### without.jpg
            
            image_path = int(number)##casted
            full_path = os.path.join(path,"images",full_path)##full image path
            
            
            if full_path[-1] == "\n":
                full_path = full_path[:-1]
                
        
            image = plt.imread(full_path)
            
            factor_h = image.shape[0]/40
            factor_w = image.shape[1]/30
            
            i=0
            for x_point,y_point in zip(category[image_path-1,:,0],category[image_path-1,:,1]):
                
                if category[image_path-1,i,0]!=-1 and category[image_path-1,i,1]!=-1:
                    
                    category[image_path-1,i,1]=int(category[image_path-1,i,1]/factor_h)
                    category[image_path-1,i,0]=int(category[image_path-1,i,0]/factor_w)
                    
                i+=1
                
            labels.append(category[image_path-1])
            
    return np.array(labels)

In [ ]:
mat = scipy.io.loadmat(os.path.join(paths[path_index],"coords.mat"))["coords"]

mats =list()
mats.append(get_keypoints(mat,avg=True))
for i in range(3):
    mats.append(get_keypoints(mat,avg=False,annotator=i))
                  
Y_points_train = list()
Y_points_test = list()
for i in range(4):
    Y_points_train.append(get_labels(paths[path_index],mats[i]))
    Y_points_test.append(get_labels(paths[path_index],mats[i],train=False))

In [ ]:
train_labels = np.array(Y_points_train)
test_labels = np.array(Y_points_test) 
print(train_labels.shape)
print(test_labels.shape)

In [ ]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
X_train1 = get_data(paths[path_index],size=(240,320))
X_train2 = get_data(paths[path_index],size=(120,160))
X_train3 = get_data(paths[path_index],size=(60,80))
X_train = [X_train1,X_train2,X_train3]

X_test1 = np.array(get_data(paths[path_index],size=(240,320),train=False))
X_test2 = np.array(get_data(paths[path_index],size=(120,160),train=False))
X_test3 = np.array(get_data(paths[path_index],size=(60,80),train=False))
X_test=[X_test1,X_test2,X_test3]

# **HeatMaps**

---



In [ ]:
# def clustering_heatmap(data):
#   def get_mid(x):
#     i,mid,coordinates = x
#     return mid
#   keypoints = data.shape[-2]
#   left = [] ## array with indices of midpoint less than mid width of image
#   right = []## array with indices of midpoint greater than mid width of image

 
#   for i in range(data.shape[0]):
#     coordinates = data[i].copy()
#     ### translate_coords with respect to 8,9
#     xt1,_= coordinates[8]
#     xt2,_ = coordinates[9]
#     mid_chair = int((xt1+xt2)/2)
#     translation_value = int(HEATMAP_SIZE[1]/2)-mid_chair

#     coordinates[:,0] +=translation_value 
#     x1,_ = coordinates[0]
#     x2,_ = coordinates[1]
#     coordinates[:,0] -=translation_value
#     mid = int(round((x1+x2)/2))
#     if mid>int(HEATMAP_SIZE[1]/2):
#       right.append((i,mid,coordinates))
#     else:
#       left.append((i,mid,coordinates))
  
#   left.sort(key=get_mid)
#   right.sort(key=get_mid)

#   ## left and right is distributed into four quarters
#   left_distribution_heatmap = np.zeros((HEATMAP_SIZE[0],HEATMAP_SIZE[1],keypoints),dtype=np.float32)
#   right_distribution_heatmap = np.zeros((HEATMAP_SIZE[0],HEATMAP_SIZE[1],keypoints),dtype=np.float32)
#   for item in left:
#     _,_,coordinates = item
#     for i,coord in enumerate(coordinates):
#       x,y = coord
#       while x >= HEATMAP_SIZE[1]:
#         x-=1
#       while y >= HEATMAP_SIZE[0]:
#         y-=1
#       left_distribution_heatmap[y,x,i]+=5

#   for item in right:
#     _,_,coordinates = item
#     for i,coord in enumerate(coordinates):
#       x,y = coord
#       while x >= HEATMAP_SIZE[1]:
#         x-=1
#       while y >= HEATMAP_SIZE[0]:
#         y-=1
#       left_distribution_heatmap[y,x,i]+=5

#   left_distribution_heatmap = ((left_distribution_heatmap-left_distribution_heatmap.min())
#                                 /(left_distribution_heatmap.max()-left_distribution_heatmap.min()))*0.25
    
  
#   right_distribution_heatmap = ((right_distribution_heatmap-right_distribution_heatmap.min())
#                                 /(right_distribution_heatmap.max()-right_distribution_heatmap.min()))*0.25


#   return [left,right]

In [ ]:
# left,right = clustering_heatmap(train_labels[annotator])
# print(len(left))
# print(len(right))
# for i in range(750,799):
#   index,mid,coordinates = left[i]
#   plt.figure(figsize=(8,8))
#   plt.imshow(X_train[0][index])

In [ ]:
# ### back&front chairs could be solved by slopes
# for i in range(900,949):
#   index,mid,coordinates = right[i]
#   plt.figure(figsize=(8,8))
#   heatmap = np.zeros((HEATMAP_SIZE[0],HEATMAP_SIZE[1]))
#   heatmap[coordinates[0,1]-1,mid+1] = 255
#   heatmap = tf.image.resize(np.expand_dims(heatmap,axis=-1),[X_train[0][index].shape[0],X_train[0][index].shape[1]])
#   plt.imshow(X_train[0][index]+heatmap)

In [ ]:
def scoring_sum(labels,keypoints_num):
    heatmaps = list()
    for example in range(labels[0].shape[0]):
        main_heatmap = np.zeros(shape=(40,30,keypoints_num),dtype="float32")
        for label in labels:
            i=0
            for x,y in zip(label[example,:,0],label[example,:,1]):
                if x!=-1 and y!=-1 and x<=29 and y<=39:
                  main_heatmap[y,x,i]+=5
                i+=1      
        norm = (main_heatmap-main_heatmap.min())/(main_heatmap.max()-main_heatmap.min())                
        heatmaps.append(norm)
        
    return np.array(heatmaps)

In [ ]:
def normal_heatmap(data,kp_ct):
  keypoints = data.shape[-2]
  heatmaps = []
  for i in range(data.shape[0]):
    heatmap = np.zeros(shape=(HEATMAP_SIZE[0],HEATMAP_SIZE[1],kp_ct),dtype=np.float32)
    coordinates = data[i]
    k = 0
    for x,y in coordinates:
      if k == kp_ct:
        break
      if x>=0 and y>=0 and x<=HEATMAP_SIZE[1] and y<=HEATMAP_SIZE[0]:
        if x == HEATMAP_SIZE[1]:
          x-=1
        if y == HEATMAP_SIZE[0]:
          y-=1
        heatmap[y,x,k] = 1
      k+=1
    heatmaps.append(heatmap)
  
  heatmaps = np.array(heatmaps)
  normalized = (heatmaps-heatmaps.min())/(heatmaps.max()-heatmaps.min())
  return heatmaps

In [ ]:
def gaussian_heatmap(data,sigma=1.0,key_points=-1,on_point=False):

  Z = 1/(2 * np.pi * sigma ** 2)
  if key_points == -1:
    key_points = data.shape[-2]

  heatmaps = []  
  
  for example in data:
    midy = int(HEATMAP_SIZE[0]/2) 
    midx = int(HEATMAP_SIZE[1]/2)

    k=0
    heatmap = np.zeros((HEATMAP_SIZE[0],HEATMAP_SIZE[1],key_points),dtype=np.float32)
    for key_point in example:
      if k == key_points and key_points!=-1:
        break
      x,y = key_point
      if x == HEATMAP_SIZE[1]:
        x-=1
      if y == HEATMAP_SIZE[0]:
        y-=1
      if on_point:
        ## AS WE GO FURTHER DECREASE SCORE ## 
        midx,midy = x,y
        if midx>0 and midy>0:
          for row in range(1,HEATMAP_SIZE[0]):
            for column in range(1,HEATMAP_SIZE[1]):
              c = -((column-midx)**2 + (row-midy)**2)
              heatmap[row,column,k] = Z*np.exp(c/(2*sigma**2))
        
      else:
        ## USE CENTER OF IMAGE APPROACH ## 
        if x>=0 and y>=0 and x<=HEATMAP_SIZE[1] and y<=HEATMAP_SIZE[0]:
          c = -((x-midx)**2 + (y-midy)**2)
          heatmap[y,x,k] = Z*np.exp(c/(2*sigma**2))
      heatmap[0,0,k] = 0
      k+=1
    heatmap/=np.sum(heatmap)
    heatmaps.append(heatmap)

  heatmaps = np.array(heatmaps)
  normalized = (heatmaps-heatmaps.min())/(heatmaps.max()-heatmaps.min())
  return normalized

In [ ]:
def visualize_heatmap(images,map,index,key_point,all=False,resize=False):
  """
  Args:
    image: an array of (B,W,H,C)
    map: an array of (B,W,H,K) where key is the number of key-points
    index: an index for the image
    key_point: an index for the key_point
  """

  heatmap = None
  image = images[index]

  if all:
    ## Generating complete heatmap
    ### when generating all use non-max_suppression
    ### make sure each keypoint has constant intensity value
    complete_map = np.zeros((map.shape[1],map.shape[2]))

    for i in range(map.shape[-1]):
      minimap = map[index,:,:,i].copy()

      max_value = minimap.max()
      
      minimap[minimap<max_value]=0
      r = tf.argmax(tf.argmax(minimap,axis=1))
      c = tf.argmax(tf.argmax(minimap,axis=0))
      minimap[r,c] = 1.0
      complete_map+=minimap

    heatmap = complete_map
  else:
    heatmap = map[index,:,:,key_point]
  
  if resize:
    heatmap = np.expand_dims(heatmap,-1)
    heatmap = tf.image.resize(heatmap,[image.shape[0],image.shape[1]])
    heatmap = np.squeeze(heatmap,axis=-1)

  fig, axes = plt.subplots(1, 2, figsize=(8, 4))
  ax = axes.ravel()

  ax[0].imshow(image)
  ax[0].set_title("Original")

  ax[1].imshow(image, vmin=0, vmax=2)
  ax[1].imshow(heatmap, alpha=0.8,cmap="gray")
  ax[1].set_title("Original + heatmap")

  fig.tight_layout()
  plt.show()
  plt.figure(figsize=(20,20))

In [ ]:
def rotate_randomly(X_train,heatmaps):
  shape = len(X_train[0])
  scales = dict()

  for i in range(3):
    scales[i] = []

  r_heatmaps = []
  for i in range(shape-1,-1,-1):
    rotation_angle = random.randint(-1,1)#tf.random.uniform([1],minval=-5,maxval=5,dtype=tf.int32)
    probability = tf.random.uniform([1],minval=0,maxval=1)[0] 

    if probability > 0.9 and (rotation_angle<0 or rotation_angle>0):

      rotation_angle = tf.cast((rotation_angle/10),tf.float32)
      for k in range(3):
        scale = tfa.image.rotate(X_train[k][i],rotation_angle,interpolation="bilinear")
        scales[k].append(scale)

      rotated_heatmap = tfa.image.rotate(heatmaps[i],rotation_angle,interpolation="bilinear")
      r_heatmaps.append(rotated_heatmap)

    elif probability > 0.85:
      for k in range(3):
        scale = tf.image.flip_left_right(X_train[k][i])
        scales[k].append(scale)

      flipped_heatmap = tf.image.flip_left_right(heatmaps[i])
      r_heatmaps.append(flipped_heatmap)

    if i == int(shape/4):
      print("Done 75%")
    elif i == int(shape/2):
      print("Done 50%")
    elif i == int(shape/4)*3:
      print("Done 25%")
  
  # for i in range(3):
  #   scales[i] = np.array(scales[i])

  # return scales,np.array(r_heatmaps)
  return scales,r_heatmaps

In [ ]:
heatmaps = None
test_heatmaps = None
if hmaps == "GAUSSIAN":
   heatmaps = gaussian_heatmap(train_labels[annotator],on_point=True,sigma=1.5,key_points=kp)
   test_heatmaps = gaussian_heatmap(test_labels[annotator],on_point=True,sigma=1.5,key_points=kp)
elif hmaps == "KDE":
  for i in range(2):
    if heatmaps is None:
      heatmaps = gaussian_heatmap(train_labels[i],on_point=True,sigma=1.5,key_points=kp)
      test_heatmaps = gaussian_heatmap(test_labels[i],on_point=True,sigma=1.5,key_points=kp)
    else:
      heatmaps = np.add(heatmaps,gaussian_heatmap(train_labels[i],on_point=True,sigma=1.5,key_points=kp))
      test_heatmaps = np.add(test_heatmaps,gaussian_heatmap(test_labels[i],on_point=True,sigma=1.5,key_points=kp))
      
elif hmaps == "NORMAL":
  heatmaps = normal_heatmap(train_labels[annotator],kp)
  test_heatmaps = normal_heatmap(test_labels[annotator],kp)
elif hmaps == "SCORE":
  heatmaps = scoring_sum(train_labels,kp)
  test_heatmaps = scoring_sum(test_labels,kp)
 
heatmaps = (heatmaps-heatmaps.min())/(heatmaps.max()-heatmaps.min())
test_heatmaps = (test_heatmaps-test_heatmaps.min())/(test_heatmaps.max()-test_heatmaps.min())

In [ ]:
scales,r_heatmaps = rotate_randomly(X_train,heatmaps)

In [ ]:
for i in range(len(scales[0])):
  index = random.randint(0,len(scales[0])-1)
  for k in range(3):
    X_train[k].insert(index,scales[k][i])
  
  heatmaps = np.insert(heatmaps,index,r_heatmaps[i],axis=0)

del scales,r_heatmaps
gc.collect()
del  X_train1,X_train2,X_train3,train_labels,test_labels,mats
gc.collect()

In [ ]:
for i in range(3):
  X_test[i] = np.array(X_test[i])
  X_train[i] = np.array(X_train[i])
  print(X_train[i].max())
print(heatmaps.max())
print(X_test[0].shape)

In [ ]:
print(heatmaps.shape)

In [ ]:
for i in range(40):
  visualize_heatmap(X_train[0],heatmaps,i,0,all=True,resize=True)

In [ ]:
for i in range(3):
  print(X_train[i].shape)
print(heatmaps.shape)

# **Model Building & Training**

---



In [ ]:
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Input,UpSampling2D,concatenate,Flatten,Dense,Conv2DTranspose,Cropping2D
from tensorflow.keras.utils import plot_model

In [ ]:
class ConvMax(tf.keras.Model):
  
  def __init__(self,filters,kernel_size,scale=None,last = False):
    super(ConvMax, self).__init__()
    self.conv = Conv2D(filters,kernel_size=kernel_size,activation='relu',padding="same")
    self.bn = BatchNormalization(center=True,scale=True,trainable=True)
    self.pool = MaxPooling2D(pool_size=(2, 2))
    self.last = last
    self.scale = scale

    if self.last:
     self.conv2 = Conv2D(512,kernel_size = (9,9),activation="relu",padding="same") ### replace hard coded later
     self.bn2 = BatchNormalization(center=True,scale=True,trainable=True)

  
  def call(self,input_tensor):
    x = self.conv(input_tensor)
    x = self.bn(x)
    x = self.pool(x)

    if self.last:
      x = self.conv2(x)
      x = self.bn2(x)

    if self.scale is not None:
      x = tf.image.resize(x, [40,30]) ## replace hard coded scale later

    return x

In [ ]:
class ScaleBlock(tf.keras.Model):
    def __init__(self, filters, kernel_size,scale):
        super(ScaleBlock, self).__init__()

        self.convmax1 = ConvMax(filters[0],kernel_size)
        self.convmax2 = ConvMax(filters[1],kernel_size)
        self.convmax3 = ConvMax(filters[2],kernel_size,last=True,scale=scale)
    
    def call(self, input_tensor):
      x = self.convmax1(input_tensor)
      x = self.convmax2(x)
      x = self.convmax3(x)

      return x

In [ ]:
class Model(tf.keras.Model):

  def __init__(self,filters,kernel_size,keypoints,regression_filter=512):

    super(Model, self).__init__()
    self.scaleb1 = ScaleBlock(filters,kernel_size,scale=False)
    self.scaleb2 = ScaleBlock(filters,kernel_size,scale=True)
    self.scaleb3 = ScaleBlock(filters,kernel_size,scale=True)

    self.bn1 = BatchNormalization(center=True,scale=True,trainable=True)

    self.conv1 = Conv2D(regression_filter,kernel_size=(1,1),padding="same",activation = "relu")
    self.bn2 = BatchNormalization(center=True,scale=True,trainable=True)

    self.conv2 = Conv2D(regression_filter,kernel_size=(1,1),padding="same",activation="relu")
    self.bn3 = BatchNormalization(center=True,scale=True,trainable=True)

    self.conv_key = Conv2D(keypoints,kernel_size=(1,1),padding="same",activation="sigmoid")

    self.encoder = tf.keras.Sequential(
        [
         Flatten(),
         Dense(8192,activation="relu"),
         Dense(4096,activation="relu")
        ]
     )
    self.decoder = tf.keras.Sequential(
         [
          Dense(8192,activation="relu"),
          Dense(40*30*keypoints,activation="sigmoid"),
          tf.keras.layers.Reshape((40,30,keypoints)),
         ]
     ) 
    self.check=True
    

  def call(self,input_tensor):
    x1 = self.scaleb1(input_tensor[0])
    x2 = self.scaleb2(input_tensor[1])
    x3 = self.scaleb3(input_tensor[2])

    X = tf.keras.layers.add([x1,x2,x3])
    X = self.bn1(X)

    X = self.conv1(X)
    X = self.bn2(X)

    X = self.conv2(X)
    X = self.bn3(X)

    noisy_heatmap = self.conv_key(X)
    encoded_heatmap = self.encoder(noisy_heatmap)
    soft_heatmap = self.decoder(encoded_heatmap)

    if soft_heatmap.shape[1] == 40 and soft_heatmap.shape[2] == 28:
      soft_heatmap = tf.image.resize(soft_heatmap,[40,30])

    if self.check:
      #print(soft_heatmap.shape[1],soft_heatmap.shape[2])
      self.check = False
      assert soft_heatmap.shape[1] == 40 and soft_heatmap.shape[2] == 30

  
    return soft_heatmap

In [ ]:
def avg_error(y_true,y_pred):
  key_points_error = []

  for i in range(y_pred.shape[-1]):
    key_points_error.append(0)

  key_points_error = np.array(key_points_error)

  for i in range(y_pred.shape[0]):
    prediction = y_pred[i]
    ground_truth = y_true[i]

    for k in range(prediction.shape[-1]):
      pred_map = prediction[:,:,k].numpy()
      truth_map = ground_truth[:,:,k].numpy()

      r_pred,c_pred = np.unravel_index(pred_map.argmax(),pred_map.shape)
      r_true,c_true = np.unravel_index(truth_map.argmax(),truth_map.shape)
      
      dy = tf.abs(r_true-r_pred)
      dx = tf.abs(c_true-c_pred)
      error = dx+dy
      key_points_error[k]+=error

  avg_error = key_points_error/y_pred.shape[-1]
  return (tf.reduce_sum(avg_error)/y_pred.shape[0]) ## average error of all key_points

In [ ]:
# #@tf.function ### annotaion makes code run in graph mode
# def train_step(model,optimizer,loss_object,images,hmaps):
#   with tf.GradientTape() as tape:
#     predicted_heatmaps = model(images,training = True)
#     loss = loss_object(hmaps,predicted_heatmaps)## batch_loss
#   gradients = tape.gradient(loss,model.trainable_variables) ## derivative of loss with respect to model parameters ### dloss/dparam
#   optimizer.apply_gradients(zip(gradients,model.trainable_variables)) ## updating model weights

#   #error = avg_error(heatmaps,predicted_heatmaps)

#   return loss.numpy().mean()#,error]

In [ ]:
# def get_batch(images,heatmaps,batch_size):
#   scale1,scale2,scale3,hmp = shuffle(images[0],images[1],images[2],heatmaps)
#   number_of_batches = int(hmp.shape[0]/batch_size)
#   start = 0
#   end = batch_size
#   for i in range(number_of_batches):
#     training_data = [scale1[start:end],scale2[start:end],scale3[start:end]]
#     training_heatmaps = hmp[start:end]
#     yield (training_data,training_heatmaps)
#     start+=batch_size
#     end += batch_size

In [ ]:
# loss_history = []
# val_loss_history = []

# model = Model([64,128,256],(5,5),kp,regression_filter=512)
# optimizer = tf.keras.optimizers.Adam()
# loss_object = tf.keras.losses.BinaryCrossentropy()

In [ ]:
# for i in range(epochs):
#   ### get random batches ###
#   for batch_number, (batch_img, batch_hmp) in enumerate(get_batch(X_train,heatmaps,14)):
#     train_loss = train_step(model,optimizer,loss_object,batch_img,batch_hmp) ## mean of batch training loss
#     loss_history.append(train_loss)
#   print("last batch loss for epoch number {0}  is {1}".format(i,train_loss))


In [ ]:
### you could avoid using keras by using the training loop above (takes more memory & better with colab pro) ###
model = Model([64,128,256],(5,5),kp,regression_filter=512)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=tf.keras.losses.binary_crossentropy,metrics=[avg_error],run_eagerly=True)

In [ ]:
history = model.fit(X_train,heatmaps,epochs=33,verbose=True,batch_size=14,
                    validation_data=(X_test, test_heatmaps))

# **Model Evaluation & Inspecting Results**

---



In [ ]:
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.plot(history.history['loss'],scalex=True, scaley=True)
plt.plot(history.history['val_loss'],scalex=True, scaley=True)
plt.savefig(hmaps+".pdf")
plt.show()

In [ ]:
plt.title('model error')
plt.ylabel('error')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.plot(history.history['avg_error'],scalex=True, scaley=True)
plt.plot(history.history['val_avg_error'],scalex=True, scaley=True)
plt.savefig(hmaps+".pdf")
plt.show()

In [ ]:
key_points_error = []

for i in range(heatmaps.shape[-1]):
  key_points_error.append(0)

key_points_error = np.array(key_points_error)


for i in range(X_test[0].shape[0]):
  prediction = model.predict([np.expand_dims(X_test[0][i],axis=0),
                              np.expand_dims(X_test[1][i],axis=0),
                              np.expand_dims(X_test[2][i],axis=0)])#[1]

  t_map = test_heatmaps[i,:,:]
  for k in range(prediction.shape[-1]):
    pred_map = prediction[0,:,:,k].copy()
    truth_map = t_map[:,:,k].copy()

    r_pred,c_pred = np.unravel_index(pred_map.argmax(),pred_map.shape)
    r_true,c_true = np.unravel_index(truth_map.argmax(),truth_map.shape)
    
    dy = tf.abs(r_true-r_pred)
    dx = tf.abs(c_true-c_pred)
    
    error = dx+dy
    key_points_error[k]+=error
    

In [ ]:
%matplotlib inline
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
key_point_index = [i for i in range(heatmaps.shape[-1])]
#print(key_points_error)
ax.bar(key_point_index,key_points_error/X_test[0].shape[0])

plt.savefig(hmaps+"AE.jpg",bbox_inches='tight',dpi=150)
plt.show()

In [ ]:
key_points_error = []
for k in range(10):
  thresh=[]
  for i in range(heatmaps.shape[-1]):
    thresh.append(0)
  key_points_error.append(thresh)

key_points_error = np.array(key_points_error)


for i in range(X_test[0].shape[0]):
  prediction = model.predict([np.expand_dims(X_test[0][i],axis=0),
                              np.expand_dims(X_test[1][i],axis=0),
                              np.expand_dims(X_test[2][i],axis=0)])
  
  t_map = test_heatmaps[i,:,:]
  for k in range(prediction.shape[-1]):
    pred_map = prediction[0,:,:,k].copy()
    truth_map = t_map[:,:,k].copy()

    r_pred,c_pred = np.unravel_index(pred_map.argmax(),pred_map.shape)
    r_true,c_true = np.unravel_index(truth_map.argmax(),truth_map.shape)
    
    dy = tf.abs(r_true-r_pred)
    dx = tf.abs(c_true-c_pred)
    error = dx+dy

    for thresh in range(10):
      if error<=thresh+1:
        key_points_error[thresh][k]+=1
key_points_error = np.array(key_points_error)

In [ ]:
plt.plot([i+1 for i in range(10)],(key_points_error[:,5]/X_test[0].shape[0]))
plt.xlabel("threshold")
plt.ylabel("PCK")
plt.savefig(hmaps+"PCK.jpg",bbox_inches='tight',dpi=600)
plt.show()
#plt.plot([i+1 for i in range(10)],(key_points_error[:,1]/X_test[0].shape[0]))
#plt.plot([i+1 for i in range(10)],(key_points_error[:,2]/X_test[0].shape[0]))

In [ ]:
start = 30
end = start+64
prediction = model.predict([X_test[0][start:end],X_test[1][start:end],X_test[2][start:end]])#[1]
print(prediction.shape)
print(X_test[0][start:end].shape)

In [ ]:
for i in range(20,50):
  visualize_heatmap(X_test[0][start:end],prediction,i,-1,all=True,resize=True)

# **Building Reconstruction Model**

---



In [ ]:
if path_index == 0:
  Y_train = Y_points_train[0][:,:kp].copy().astype(np.float32)##(B,10,2)
  Y_test = Y_points_test[0][:,:kp].copy().astype(np.float32)

  Y_train[:,0] = Y_points_train[0][:,1].copy()
  Y_train[:,1] = Y_points_train[0][:,6].copy()
  Y_train[:,2] = Y_points_train[0][:,5].copy()
  Y_train[:,3] = Y_points_train[0][:,0].copy()
  Y_train[:,4] = Y_points_train[0][:,3].copy()
  Y_train[:,5] = Y_points_train[0][:,8].copy()
  Y_train[:,6] = Y_points_train[0][:,7].copy()
  Y_train[:,7] = Y_points_train[0][:,2].copy()
  Y_train[:,8] = Y_points_train[0][:,4].copy()



  Y_test[:,0] = Y_points_test[0][:,1].copy()
  Y_test[:,1] = Y_points_test[0][:,6].copy()
  Y_test[:,2] = Y_points_test[0][:,5].copy()
  Y_test[:,3] = Y_points_test[0][:,0].copy()
  Y_test[:,4] = Y_points_test[0][:,3].copy()
  Y_test[:,5] = Y_points_test[0][:,8].copy()
  Y_test[:,6] = Y_points_test[0][:,7].copy()
  Y_test[:,7] = Y_points_test[0][:,2].copy()
  Y_test[:,8] = Y_points_test[0][:,4].copy()
  
elif path_index == 2:
  Y_train = Y_points_train[0][:,:kp].copy().astype(np.float32)##(B,10,2)
  Y_test = Y_points_test[0][:,:kp].copy().astype(np.float32)

  Y_train[:,0] = Y_points_train[0][:,1].copy()
  Y_train[:,1] = Y_points_train[0][:,8].copy()
  Y_train[:,2] = Y_points_train[0][:,7].copy()
  Y_train[:,3] = Y_points_train[0][:,0].copy()
  Y_train[:,4] = Y_points_train[0][:,3].copy()
  Y_train[:,5] = Y_points_train[0][:,10].copy()
  Y_train[:,6] = Y_points_train[0][:,9].copy()
  Y_train[:,7] = Y_points_train[0][:,2].copy()
  Y_train[:,8] = Y_points_train[0][:,6].copy()
  Y_train[:,9] = Y_points_train[0][:,13].copy()
  Y_train[:,10] = Y_points_train[0][:,4].copy()
  Y_train[:,11] = Y_points_train[0][:,5].copy()
  Y_train[:,12] = Y_points_train[0][:,11].copy()
  Y_train[:,13] = Y_points_train[0][:,12].copy()


  Y_test[:,0] = Y_points_test[0][:,1].copy()
  Y_test[:,1] = Y_points_test[0][:,8].copy()
  Y_test[:,2] = Y_points_test[0][:,7].copy()
  Y_test[:,3] = Y_points_test[0][:,0].copy()
  Y_test[:,4] = Y_points_test[0][:,3].copy()
  Y_test[:,5] = Y_points_test[0][:,10].copy()
  Y_test[:,6] = Y_points_test[0][:,9].copy()
  Y_test[:,7] = Y_points_test[0][:,2].copy()
  Y_test[:,8] = Y_points_test[0][:,6].copy()
  Y_test[:,9] = Y_points_test[0][:,13].copy()
  Y_test[:,10] = Y_points_test[0][:,4].copy()
  Y_test[:,11] = Y_points_test[0][:,5].copy()
  Y_test[:,12] = Y_points_test[0][:,11].copy()
  Y_test[:,13] = Y_points_test[0][:,12].copy()
else:
  Y_train = Y_points_train[0].copy().astype(np.float32)
  Y_test = Y_points_test[0].copy().astype(np.float32)
### translate points to center then normalize

translation_value = 0
test_translation_value = 0

if path_index == 0:
  xt1 = Y_train[:,2,0]
  xt2 = Y_train[:,8,0]
  mid_obj = (xt1+xt2)/2
  mid_obj = mid_obj.astype("int32")
  translation_value = np.expand_dims(int(HEATMAP_SIZE[1]/2)-mid_obj,axis=-1)
  
  xt1 = Y_test[:,2,0]
  xt2 = Y_test[:,8,0]
  mid_obj = (xt1+xt2)/2
  mid_obj = mid_obj.astype("int32")
  test_translation_value = np.expand_dims(int(HEATMAP_SIZE[1]/2)-mid_obj,axis=-1)

elif path_index == 1:
  xt1 = Y_train[:,8,0]
  xt2 = Y_train[:,9,0]
  mid_obj = (xt1+xt2)/2
  mid_obj = mid_obj.astype("int32")
  translation_value = np.expand_dims(int(HEATMAP_SIZE[1]/2)-mid_obj,axis=-1)

  xt1 = Y_test[:,8,0]
  xt2 = Y_test[:,9,0]
  mid_obj = (xt1+xt2)/2
  mid_obj = mid_obj.astype("int32")
  test_translation_value = np.expand_dims(int(HEATMAP_SIZE[1]/2)-mid_obj,axis=-1)


# Y_train[:,:,0] +=translation_value 

Y_train[:,:,0]/=15
Y_train[:,:,0]-=1

Y_train[:,:,1]/=20
Y_train[:,:,1]-=1




# Y_test[:,:,0] += test_translation_value 
Y_test[:,:,0]/=15
Y_test[:,:,0]-=1

Y_test[:,:,1]/=20
Y_test[:,:,1]-=1

print("minimum in normalized X train is {}".format(Y_train[:,:,0].min()))
print("maximum in normalized X train is {}".format(Y_train[:,:,0].max()))

print("minimum in normalized Y train is {}".format(Y_train[:,:,1].min()))
print("maximum in normalized Y train is {}".format(Y_train[:,:,1].max()))

print("minimum in normalized X test is {}".format(Y_test[:,:,0].min()))
print("maximum in normalized X test is {}".format(Y_test[:,:,0].max()))

print("minimum in normalized Y test is {}".format(Y_test[:,:,1].min()))
print("maximum in normalized Y test is {}".format(Y_test[:,:,1].max()))

Y_train = Y_train.reshape(-1,kp*2)
Y_test = Y_test.reshape(-1,kp*2)
print(Y_train.shape)
print(Y_test.shape)

In [ ]:
def inspect_points(points,category):
  line_thickness = 1
  edges = dict()

  edges[0] = [[1,5],[2,6],[3,7],[4,8],[3,4],[5,6],[6,7],[7,8],[5,8],[8,9],[7,10],[9,10]]
  edges[1] = [[1,5],[2,6],[3,7],[4,8],[5,6],[6,7],[7,8],[5,8],[8,9],[7,10],[9,10]]
  edges[2] = [[1,5],[2,6],[3,7],[4,8],[5,6],[6,7],[7,8],[5,8],[8,9],[7,10],[9,10],[11,12],[12,5],[13,14],[14,6]]
  edges[3] = [[1,6],[2,6],[3,6],[4,6],[5,6],[6,7],[8,9],[9,10],[8,11],[10,11],[11,12],[12,13],[10,13]] 
  #edges[4] =

  if points.shape[-1]!=3:
    x = points.reshape(kp,2).copy()
  else:
    x = points.reshape(kp,3).copy()
  
  image  = np.full((40,30,3),255,dtype=np.float32)## creation of white image
  
  print(x.shape)
  x[:,0]+=1
  x[:,0]*=15
  x[:,1]+=1
  x[:,1]*=20

  for edge in edges[category]:
      p1 = x[edge[0]-1]
      p2 = x[edge[1]-1]
      
      cv.line(image, (int(p1[0]), int(p1[1])), (int(p2[0]), int(p2[1])), (0, 0, 0), thickness=line_thickness)


  plt.imshow(image,cmap="gray")
  return x

In [ ]:
index = 0
inspect_points(Y_test[index],path_index)
plt.figure()
plt.imshow(X_test[0][index])

In [ ]:
def inspect(weights,rotations,focus,to_rad=True,category=0):
  deformations = list()
  scale = 1.2
  def rotate(theta,axis):
    rad = theta
    if to_rad:
      rad = (theta/180)*math.pi

    rotation = {}
    rotation[0] = [[1,0,0,0],
                  [0,math.cos(rad),-math.sin(rad),0],
                  [0,math.sin(rad),math.cos(rad),0],
                  [0,0,0,1]]

    rotation[1] = [[math.cos(rad),0,math.sin(rad),0],
                  [0,1,0,0],
                  [-math.sin(rad),0,math.cos(rad),0],
                  [0,0,0,1]]

    rotation[2] = [[math.cos(rad),-math.sin(rad),0,0],
                  [math.sin(rad),math.cos(rad),0,0],
                  [0,0,1,0],
                  [0,0,0,1]]

    return np.array(rotation[axis],dtype=np.float32)
  
  if category == 0:
    mean_shape =  np.array([[-1,-1,2],
                           [ 1,-1,2],
                           [1,-1,-2],
                           [-1,-1,-2],
                           [-1,0,2],
                           [1,0,2], 
                           [1,0,-2], 
                           [-1,0,-2],
                           [-1,1,-2],
                           [1,1,-2]],dtype=np.float32)

    edges = [[1,5],[2,6],[3,7],[4,8],[3,4],[5,6],[6,7],[7,8],[5,8],[8,9],[7,10],[9,10]]
    d0 = np.array([[0,0,1],[0,0,1],[0,0,-1],[0,0,-1],[0,0,1],[0,0,1],[0,0,-1],[0,0,-1],[0,0,-1],[0,0,-1]])
    d1 = np.array([[-1,0,0],[1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0]])
    d2 = np.array([[0,-1,0],[0,-1,0],[0,-1,0],[0,-1,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
    d3 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,1,0],[0,1,0]])

    deformations.append(d0)
    deformations.append(d1)
    deformations.append(d2)
    deformations.append(d3)

  elif category == 1:
    mean_shape = np.array([[-1,-2,1],
                          [1,-2,1],
                          [1,-2,-1],
                          [-1,-2,-1],
                          [-1,0,1],
                          [1,0,1],
                          [1,0,-1],
                          [-1,0,-1],
                          [-1,2,-1],
                          [1,2,-1]],dtype=np.float32)
    
    edges = [[1,5],[2,6],[3,7],[4,8],[5,6],[6,7],[7,8],[5,8],[8,9],[7,10],[9,10]]
    d0 = np.array([[0,-1,0],[0,-1,0],[0,-1.,0],[0,-1,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0], [0,0,0],[0,0,0]])
    d1 =  np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0.,0],[0,0,0],[0,1,0],[0,1,0]])
    d2 = np.array([[-1.,0,0],[1.,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0]])
    d3 =  np.array([[-1.,0,1],[1,0,1],[1,0,-1],[-1.,0,-1],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])

    deformations.append(d0)
    deformations.append(d1)
    deformations.append(d2)
    deformations.append(d3)

  elif category == 2:
    mean_shape = np.array([[-2,-2,1],
                          [2,-2,1],
                          [2,-2,-1],
                          [-2,-2,-1],
                          [-2,0,1], 
                          [2,0,1],
                          [2,0,-1],
                          [-2,0,-1],
                          [-2,2,-1],
                          [2,2,-1],
                          [-2,1,-1],
                          [-2,1,1],
                          [2,1,-1],
                          [2,1,1]],dtype=np.float32)
    
    edges = [[1,5],[2,6],[3,7],[4,8],[5,6],[6,7],[7,8],[5,8],[8,9],[7,10],[9,10],[11,12],[12,5],[13,14],[14,6]]
    d0 = np.array([[0,0,1],[0,0,1],[0,0,-1],[0,0,-1],[0,0,1],[0,0,1],[0,0,-1],[0,0,-1],[0,0,-1],[0,0,-1],[0,0,-1],[0,0,1],[0,0,-1],[0,0,1]])
    d1 = np.array([[-1,0,0],[1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0],[1,0,0]])
    d2 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,1,0],[0,1,0],[0,0.5,0],[0,0.5,0],[0,0.5,0],[0,0.5,0]])
    d3 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,1,0],[0,1,0],[0,1,0],[0,1,0]])
    d4 = np.array([[0,-1,0],[0,-1,0],[0,-1,0],[0,-1,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
    d5 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,-1],[0,0,0],[0,0,-1]])

    deformations.append(d0)
    deformations.append(d1)
    deformations.append(d2)
    deformations.append(d3)
    deformations.append(d4)
    deformations.append(d5)

  elif category == 3:
    mean_shape = np.array([[0.309*scale,-1.5,0.9511*scale],
                           [-0.809*scale,-1.5,0.5878*scale],
                           [-0.809*scale,-1.5,-0.5878*scale], 
                           [0.309*scale,-1.5,-0.9511*scale],
                           [1*scale,-1.5,0.0000*scale],
                           [0,-1.5,0],
                           [0,0,0],
                           [-1,0,1],
                           [1,0,1],
                           [1,0,-1],
                           [-1,0,-1],
                           [-1,1.5,-1],
                           [1,1.5,-1]],dtype=np.float32)
   
    edges = [[1,6],[2,6],[3,6],[4,6],[5,6],[6,7],[8,9],[9,10],[8,11],[10,11],[11,12],[12,13],[10,13]]
    d0 = np.array([[0,-1,0],[0,-1,0],[0,-1,0],[0,-1,0],[0,-1,0],[0,-1,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
    d1 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,1,0],[0,1,0]])
    d2 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[-1,0,0],[1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0]])
    d3 = np.array([[0.309,0,0.9511],[-0.809,0,0.5878],[-0.809,0,-0.5878],[0.309,0,-0.9511],[1,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
    d4 = np.array([[-0.9511,0,0.309],[-0.5878,0,-0.809],[0.5878,0,-0.809],[0.9511,0,0.309],[0,0,1],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
    d5 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,1,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])

    deformations.append(d0)
    deformations.append(d1)
    deformations.append(d2)
    deformations.append(d3)
    deformations.append(d4)
    deformations.append(d5)

  line_thickness = 1
  image  = np.full((40,30,3),255,dtype=np.float32)## creation of white image

  rx,ry,rz = rotations
  #tx,ty,tz = translations
  weighted_shape = mean_shape.copy()
  for i in range(len(deformations)):
    weighted_shape+=(deformations[i]*weights[i])


  mean_shape = weighted_shape.copy()
  
  points = mean_shape.T
  ones = np.expand_dims(np.ones(points.shape[-1]),axis=0)
  points = np.concatenate([points,ones],axis=0)

  Rx = rotate(rx,axis=0) ## X-axis
  Ry = rotate(ry,axis=1) ## Y-axis
  Rz = rotate(rz,axis=2) ## Z-axis
  Rxy = np.dot(Rx,Ry)## 

  R = np.dot(Rxy,Rz)### rotation matrix

  points = np.dot(R,points) ## Rx+T

  points[2]-=10

  points = points/((1/focus)*points[2]+1)

  points[0,:]+=1
  points[0,:]*=15
  points[1,:]+=1
  points[1,:]*=20
  points = points.T

  for edge in edges:
      p1 = points[edge[0]-1]
      p2 = points[edge[1]-1]
      
      cv.line(image, (round(p1[0]), round(p1[1])), (round(p2[0]), round(p2[1])), (0, 0, 0), thickness=line_thickness)

  plt.imshow(image,cmap="gray")
  return points.astype("int32")

In [ ]:
inspect([0,0,0,0,0,0,0],[40,80,0],2,category=path_index)

In [ ]:
def pj_error_wrapper(kp,deform_count,category=0):
  def projection_error(y_true,y_pred):
    
    deformations = list()

    if category == 0:
      mean_shape =  np.array([[-1,-1,2],
                           [ 1,-1,2],
                           [1,-1,-2],
                           [-1,-1,-2],
                           [-1,0,2],
                           [1,0,2], 
                           [1,0,-2], 
                           [-1,0,-2],
                           [-1,1,-2],
                           [1,1,-2]],dtype=np.float32)

      d0 = np.array([[0,0,1],[0,0,1],[0,0,-1],[0,0,-1],[0,0,1],[0,0,1],[0,0,-1],[0,0,-1],[0,0,-1],[0,0,-1]])
      d1 = np.array([[-1,0,0],[1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0]])
      d2 = np.array([[0,-1,0],[0,-1,0],[0,-1,0],[0,-1,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
      d3 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,1,0],[0,1,0]])

      deformations.append(d0)
      deformations.append(d1)
      deformations.append(d2)
      deformations.append(d3)

    elif category == 1:
      mean_shape = np.array([[-1,-2,1],
                            [1,-2,1],
                            [1,-2,-1],
                            [-1,-2,-1],
                            [-1,0,1],
                            [1,0,1],
                            [1,0,-1],
                            [-1,0,-1],
                            [-1,2,-1],
                            [1,2,-1]],dtype=np.float32)

      d0 = np.array([[0,-1,0],[0,-1,0],[0,-1.,0],[0,-1,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0], [0,0,0],[0,0,0]])
      d1 =  np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0.,0],[0,0,0],[0,1,0],[0,1,0]])
      d2 = np.array([[-1.,0,0],[1.,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0]])
      d3 =  np.array([[-1.,0,1],[1,0,1],[1,0,-1],[-1.,0,-1],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])

      deformations.append(d0)
      deformations.append(d1)
      deformations.append(d2)
      deformations.append(d3)
      
    elif category == 2:
      mean_shape = np.array([[-2,-2,1],
                          [2,-2,1],
                          [2,-2,-1],
                          [-2,-2,-1],
                          [-2,0,1], 
                          [2,0,1],
                          [2,0,-1],
                          [-2,0,-1],
                          [-2,2,-1],
                          [2,2,-1],
                          [-2,1,-1],
                          [-2,1,1],
                          [2,1,-1],
                          [2,1,1]],dtype=np.float32)
      
      d0 = np.array([[0,0,1],[0,0,1],[0,0,-1],[0,0,-1],[0,0,1],[0,0,1],[0,0,-1],[0,0,-1],[0,0,-1],[0,0,-1],[0,0,-1],[0,0,1],[0,0,-1],[0,0,1]])
      d1 = np.array([[-1,0,0],[1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0],[1,0,0]])
      d2 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,1,0],[0,1,0],[0,0.5,0],[0,0.5,0],[0,0.5,0],[0,0.5,0]])
      d3 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,1,0],[0,1,0],[0,1,0],[0,1,0]])
      d4 = np.array([[0,-1,0],[0,-1,0],[0,-1,0],[0,-1,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
      d5 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,-1],[0,0,0],[0,0,-1]])

      deformations.append(d0)
      deformations.append(d1)
      deformations.append(d2)
      deformations.append(d3)
      deformations.append(d4)
      deformations.append(d5)

    elif category == 3:
      scale=1.2
      mean_shape = np.array([[0.309*scale,-1.5,0.9511*scale],
                            [-0.809*scale,-1.5,0.5878*scale],
                            [-0.809*scale,-1.5,-0.5878*scale], 
                            [0.309*scale,-1.5,-0.9511*scale],
                            [1*scale,-1.5,0.0000*scale],
                            [0,-1.5,0],
                            [0,0,0],
                            [-1,0,1],
                            [1,0,1],
                            [1,0,-1],
                            [-1,0,-1],
                            [-1,1.5,-1],
                            [1,1.5,-1]],dtype=np.float32)
    
      edges = [[1,6],[2,6],[3,6],[4,6],[5,6],[6,7],[8,9],[9,10],[8,11],[10,11],[11,12],[12,13],[10,13]]
      d0 = np.array([[0,-1,0],[0,-1,0],[0,-1,0],[0,-1,0],[0,-1,0],[0,-1,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
      d1 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,1,0],[0,1,0]])
      d2 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[-1,0,0],[1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0]])
      d3 = np.array([[0.309,0,0.9511],[-0.809,0,0.5878],[-0.809,0,-0.5878],[0.309,0,-0.9511],[1,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
      d4 = np.array([[-0.9511,0,0.309],[-0.5878,0,-0.809],[0.5878,0,-0.809],[0.9511,0,0.309],[0,0,1],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
      d5 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,1,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])

      deformations.append(d0)
      deformations.append(d1)
      deformations.append(d2)
      deformations.append(d3)
      deformations.append(d4)
      deformations.append(d5)
      
    data = y_pred
    loss=0
    for i in range(data.shape[0]):
      y_pred=data[i]
      weights,R,focus = y_pred[:deform_count],y_pred[deform_count:deform_count+9],y_pred[deform_count+9]  
      R = tf.reshape(R,[3,3])

      weighted_shape = mean_shape.copy()
      for k in range(len(deformations)):
        weighted_shape +=(weights[k]*deformations[k])
    
      points = tf.transpose(weighted_shape)

      points = tf.matmul(R,points)

      points = tf.stack([points[0],points[1],points[2]-10])
      points = points/((1/focus)*points[2]+1)##projection

      points = points[0:2]
      points = tf.transpose(points)

      points = tf.reshape(points,[kp*2])

      loss += tf.losses.mean_squared_error(y_true[i],points)#tf.keras.losses.mean_squared_logarithmic_error(y_true[i],points)#
    return loss/data.shape[0]

  return projection_error

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(46)
#9 13 20 23 28 38 43 44* 46
def construction_model(w,input=Input(shape=(20)),category=0):
  ### seed 3 -1 2 5 9 14 15 16 30 33 34
  x = Flatten()(input)
  if category == 0 or category == 3:
    x = Dense(2048,activation="relu")(x)
    x = Dense(1024,activation="relu")(x)
    x = Dense(512)(x)
    x = Dense(256)(x)
    x = Dense(128)(x)
    x = Dense(10+w)(x)

  elif category == 1:    
    x = Dense(2048,activation="relu")(x)
    x = Dense(512,activation="relu")(x)
    x = Dense(128)(x)
    x = Dense(14)(x)

  elif category == 2:
    x = Dense(2048,activation="relu")(x)
    x = Dense(512,activation="relu")(x)
    x = Dense(128)(x)
    x = Dense(16)(x)

  return tf.keras.Model(inputs=input,outputs=x)

In [ ]:
model = construction_model(w_ct,Input(shape=(kp*2)),category=path_index)
plot_model(model)
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer = optimizer,loss=pj_error_wrapper(kp,w_ct,category=path_index),run_eagerly=True)

In [ ]:
history = model.fit(Y_train,
          Y_train,
          validation_data=(Y_test,Y_test),
          epochs=30, verbose=True, batch_size=14)

# ***Construction Model Evaluation and inspecting results***

---

In [ ]:
!rm -r swivelchair_construct_weights

In [ ]:
model.save(paths[path_index]+"_construct_weights")

In [ ]:
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.axis([0,30,0,0.1])
plt.plot(history.history['loss'],scalex=True, scaley=True)
plt.plot(history.history['val_loss'],scalex=True, scaley=True)
plt.savefig(hmaps+".pdf")
plt.show()

In [ ]:
def inspect(weights,rotation,focus,refine=False,category=0):

  line_thickness = 1
  image  = np.full((80,80,3),255,dtype=np.float32)## creation of white image
  
  deformations = list()
  if category == 0:
    mean_shape =  np.array([[-1,-1,2],
                            [1,-1,2],
                            [1,-1,-2],
                            [-1,-1,-2],
                            [-1,0,2],
                            [1,0,2], 
                            [1,0,-2],
                            [-1,0,-2],
                            [-1,1,-2],
                            [1,1,-2]],dtype=np.float32)
    
    edges = [[1,5],[2,6],[3,7],[4,8],[3,4],[5,6],[6,7],[7,8],[5,8],[8,9],[7,10],[9,10]]

    d0 = np.array([[0,0,1],[0,0,1],[0,0,-1],[0,0,-1],[0,0,1],[0,0,1],[0,0,-1],[0,0,-1],[0,0,-1],[0,0,-1]])
    d1 = np.array([[-1,0,0],[1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0]])
    d2 = np.array([[0,-1,0],[0,-1,0],[0,-1,0],[0,-1,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
    d3 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,1,0],[0,1,0]])

    deformations.append(d0)
    deformations.append(d1)
    deformations.append(d2)
    deformations.append(d3)

  elif category == 1:
    mean_shape = np.array([[-1,-2,1],
                          [1,-2,1],
                          [1,-2,-1],
                          [-1,-2,-1],
                          [-1,0,1],
                          [1,0,1],
                          [1,0,-1],
                          [-1,0,-1],
                          [-1,2,-1],
                          [1,2,-1]],dtype=np.float32)
    
    edges = [[1,5],[2,6],[3,7],[4,8],[5,6],[6,7],[7,8],[5,8],[8,9],[7,10],[9,10]]

    d0 = np.array([[0,-1,0],[0,-1,0],[0,-1.,0],[0,-1,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0], [0,0,0],[0,0,0]])
    d1 =  np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0.,0],[0,0,0],[0,1,0],[0,1,0]])
    d2 = np.array([[-1.,0,0],[1.,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0]])
    d3 =  np.array([[-1.,0,1],[1,0,1],[1,0,-1],[-1.,0,-1],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])

    deformations.append(d0)
    deformations.append(d1)
    deformations.append(d2)
    deformations.append(d3)

  elif category == 2:
  
    mean_shape = np.array([[-2,-2,1],
                          [2,-2,1],
                          [2,-2,-1],
                          [-2,-2,-1],
                          [-2,0,1], 
                          [2,0,1],
                          [2,0,-1],
                          [-2,0,-1],
                          [-2,2,-1],
                          [2,2,-1],
                          [-2,1,-1],
                          [-2,1,1],
                          [2,1,-1],
                          [2,1,1]],dtype=np.float32)
    
    edges = [[1,5],[2,6],[3,7],[4,8],[5,6],[6,7],[7,8],[5,8],[8,9],[7,10],[9,10],[11,12],[12,5],[13,14],[14,6]]
    d0 = np.array([[0,0,1],[0,0,1],[0,0,-1],[0,0,-1],[0,0,1],[0,0,1],[0,0,-1],[0,0,-1],[0,0,-1],[0,0,-1],[0,0,-1],[0,0,1],[0,0,-1],[0,0,1]])
    d1 = np.array([[-1,0,0],[1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0],[1,0,0]])
    d2 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,1,0],[0,1,0],[0,0.5,0],[0,0.5,0],[0,0.5,0],[0,0.5,0]])
    d3 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,1,0],[0,1,0],[0,1,0],[0,1,0]])
    d4 = np.array([[0,-1,0],[0,-1,0],[0,-1,0],[0,-1,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
    d5 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,-1],[0,0,0],[0,0,-1]])

    deformations.append(d0)
    deformations.append(d1)
    deformations.append(d2)
    deformations.append(d3)
    deformations.append(d4)
    deformations.append(d5)

  elif category == 3:
    scale=1.2
    mean_shape = np.array([[0.309*scale,-1.5,0.9511*scale],
                          [-0.809*scale,-1.5,0.5878*scale],
                          [-0.809*scale,-1.5,-0.5878*scale], 
                          [0.309*scale,-1.5,-0.9511*scale],
                          [1*scale,-1.5,0.0000*scale],
                          [0,-1.5,0],
                          [0,0,0],
                          [-1,0,1],
                          [1,0,1],
                          [1,0,-1],
                          [-1,0,-1],
                          [-1,1.5,-1],
                          [1,1.5,-1]],dtype=np.float32)
  
    edges = [[1,6],[2,6],[3,6],[4,6],[5,6],[6,7],[8,9],[9,10],[8,11],[10,11],[11,12],[12,13],[10,13]]
    d0 = np.array([[0,-1,0],[0,-1,0],[0,-1,0],[0,-1,0],[0,-1,0],[0,-1,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
    d1 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,1,0],[0,1,0]])
    d2 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[-1,0,0],[1,0,0],[1,0,0],[-1,0,0],[-1,0,0],[1,0,0]])
    d3 = np.array([[0.309,0,0.9511],[-0.809,0,0.5878],[-0.809,0,-0.5878],[0.309,0,-0.9511],[1,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
    d4 = np.array([[-0.9511,0,0.309],[-0.5878,0,-0.809],[0.5878,0,-0.809],[0.9511,0,0.309],[0,0,1],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])
    d5 = np.array([[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,1,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]])

    deformations.append(d0)
    deformations.append(d1)
    deformations.append(d2)
    deformations.append(d3)
    deformations.append(d4)
    deformations.append(d5)

  for i in range(len(deformations)):
    mean_shape+= (weights[i]*deformations[i])

  points = mean_shape.T
  R = rotation.reshape((3,3))
  points = np.dot(R,points) ## Rx+T

  points[2]-=10
  points = points/((1/focus)*points[2]+1)

  points[0,:]+=1
  points[0,:]*=35
  points[1,:]+=1
  points[1,:]*=35
  points[0]+=5
  points[1]+=5
  points = points.T

  for edge in edges:
      p1 = points[edge[0]-1]
      p2 = points[edge[1]-1]
      
      if refine:
        if category == 1:
          # if edge == [9,10]:
          #   p1[1] = p2[1]-0.5
            # p2[1] = p
          if edge in [[7,10],[3,7],[1,5],[2,6],[8,9]]:
            p1[0] = p2[0]
            

      cv.line(image, (round(p1[0]), round(p1[1])), (round(p2[0]), round(p2[1])), (0, 0, 0), thickness=line_thickness)

  plt.imshow(image,cmap="gray")
  return points.astype("int32")

In [ ]:
model.evaluate(Y_test, Y_test, verbose=2)

In [ ]:
start=30
end = start+14
prediction = model.predict(Y_test[start:end])
print(prediction.shape)

In [ ]:
i = 3
inspect(prediction[i][0:w_ct],prediction[i][w_ct:w_ct+9],prediction[i][w_ct+9],category=path_index,refine=False)
rotation_matrix = prediction[i][w_ct:w_ct+9].reshape(3,3)

thetax = math.atan2(rotation_matrix[-1,1],rotation_matrix[-1,2]) 
thetay = math.atan2(-rotation_matrix[-1,0],math.sqrt(rotation_matrix[-1,1]**2+rotation_matrix[-1,2]**2))
thetaz = math.atan2(rotation_matrix[1,0],rotation_matrix[0,0])
thetax*=(180/math.pi)
thetay*=(180/math.pi)
thetaz*=(180/math.pi)
print(thetax)
print(thetay)
print(thetaz)
plt.figure()
plt.imshow(X_test2[start+i])

In [ ]:
!zip -r chair_construct_weights.zip chair_construct_weights

In [ ]:
!rm -f swivelchair_construct_weights.zip